## SDXL Resource Lists
**Homepage :** [SD Webui Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Resource Lists Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Resource-Lists-Guide)

### Check for Updates

**Current version : 2024.03.04**

In [ ]:
import ipywidgets as widgets
from IPython.utils import capture 
import requests

button = widgets.Button(description="Download now", button_style='success')
output = widgets.Output()

def on_button_clicked(b, url, version):
  with output:
    print('Downloading...')
    !wget -nv -O /notebooks/sdxl_resource_lists_{version}.ipynb {url}
    print(f'sdxl_resource_lists_{version}.ipynb downloaded in the root directory')

currentVersion = '2024.03.04'
updateURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main/updates.json'
res = requests.get(updateURL)

if res.status_code == 200:
    for notebook in res.json()['resources']:
        if notebook['id'] == 'sdxl':
            print('Current version : ' + currentVersion)
            print('Latest version  : ' + notebook['version'])
            if (notebook['version'] > currentVersion):
                print('\nThere is new version')
                button.on_click(lambda b: on_button_clicked(b, notebook['url'], notebook['version']))
                display(button, output)
                print('\nChangelog :')
                print(variant['changelog'])
            else:
                print('\nThis is the latest version')            
else:
    print('Failed to check for updates') 
    print('Response code :', res.status_code)

## 1. Requirements

### • Variables and Functions
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
# Civitai Token - Required to download models that require login
civitaiToken = ''


##################################### IMPORT #######################################

import os, re, requests
import ipywidgets as widgets
from IPython.utils import capture
from IPython.display import clear_output

############################ ENVIRONMENT VARIABLES ################################

os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libtcmalloc.so.4'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'
os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'

########################### GLOBAL PATHS AND FUNCTION ###########################

# Paths
root = '/notebooks'
a1111 = '/stable-diffusion-webui'
forge = '/stable-diffusion-webui-forge'

if os.path.exists(root + a1111):
    webui = root + a1111
elif os.path.exists(root + forge):
    webui = root + forge
else:
    raise Exception('Stable Diffusion Webui directory is not found')

controlNetModelsPath = webui + "/models/ControlNet"
embeddingsPath = webui + "/embeddings"
modelsPath = webui + "/models/Stable-diffusion"
loraPath = webui + "/models/Lora"
upscalerPath = webui + "/models/ESRGAN"
vaePath = webui + "/models/VAE"

# Resource URLs
mainRepoURL = 'https://raw.githubusercontent.com/ffxvs/sd-webui-complete-setup/main'
controlNetURL = mainRepoURL + '/res/sdxl/sdxl-controlnet.json'
animeModelsURL = mainRepoURL + '/res/sdxl/models/sdxl-anime-models.json'
generalModelsURL = mainRepoURL + '/res/sdxl/models/sdxl-general-models.json'
realisticModelsURL = mainRepoURL + '/res/sdxl/models/sdxl-realistic-models.json'
womanModelsURL = mainRepoURL + '/res/sdxl/models/sdxl-woman-models.json'
builtinResourcesURL = mainRepoURL + '/res/sdxl/sdxl-builtin-resources.json'
loraListURL = mainRepoURL + '/res/sdxl/sdxl-lora.json'
upscalerListURL = mainRepoURL + '/res/upscaler.json'

boolean = [False, True]
requestHeaders = {
    "Cache-Control": "no-cache, no-store, must-revalidate",
    "Pragma": "no-cache",
    "Expires": "0"
}

# Complete message
def completedMessage(): 
    completed = widgets.Button(description='Completed', button_style='success', icon='check')
    print('\n')
    display(completed)

# Resource
def getResource(url):
    res = requests.get(url, headers=requestHeaders)
    if res.status_code == 200:
        return res.json()
    else:
        return False

# Aria2c
def downloader(url, path, overwrite=False):
    args = '--download-result=hide --console-log-level=error -c -x 16 -s 16 -k 1M '
    if overwrite: args += '--allow-overwrite'
    if url.startswith('https://civitai.com/api/download/') and civitaiToken:
        if '?' in url:
            url = f'{url}&token={civitaiToken}'
        else:
            url = f'{url}?token={civitaiToken}'
    formattedURL = '"' + url + '"'
    if bool(re.search(r'\/[\w\.-]+\.\w+$', url)):
        filename = url.split('/')[-1]
        !aria2c {args} {formattedURL} -d {path} -o {filename}
    else:
        !aria2c {args} {formattedURL} -d {path}

# Git Clone
def silentClone(command, path, update=False, overwrite=False):
    directory = command.split('/')[-1]
    if os.path.exists(path + '/' + directory):
        if update:
            os.chdir(f'{path}/{directory}')
            !git pull -q
        elif overwrite:
            !rm -r {path}/{directory}
            !git clone -q --depth 10 {command} {path}/{directory}
    else:
        !git clone -q --depth 10 {command} {path}/{directory}

# WGet
def silentGet(command):
    !wget -nv {command}

# ControlNet
def downloadControlNet(controlNet, url):
    controlNetData = getResource(url)
    for model in controlNet:
        if controlNet[model]:
            print('\n' + model + '...')
            for url in controlNetData[model]:
                downloader(url, f'{controlNetModelsPath}/sdxl')
                print('')

# Model Mapper
def modelMapper(name, version, url):
    return {
        'name': name,
        'version': version,
        'url': url
    }

# Selected Models
def selectedModels(modelsURL):
    modelList = []
    for model in getResource(modelsURL)['models']:
        isSelected = eval(model['id'])
        if isSelected != 'Select version...':
            for variant in model['variants']:
                if variant['version'] == isSelected:
                    modelList.append(modelMapper(
                        model['name'],
                        variant['version'],
                        variant['url']
                    ))
    return modelList

# Download models
def downloadModels(modelsURL):
    print('⏳ Downloading selected models...')
    for model in selectedModels(modelsURL):
        print(f"\n* {model['name']} | {model['version']}")
        downloader(model['url'], f'{modelsPath}/sdxl')

# Built-in resources
def builtinResources(resourceList, resourcePath):
    for resource in resourceList:
        print(f"\n{resource['name']}...")
        downloader(resource['url'], resourcePath)

def downloadBuiltinResources():
    resource = getResource(builtinResourcesURL)
    print('⏳ Downloading built-in LoRA...')
    builtinResources(resource['lora'], f'{loraPath}/sdxl')
    print('\n\n⏳ Downloading built-in Embeddings...')
    for embedding in resource['embeddings']:
        print(embedding['name'] + '...')
        silentClone(embedding['url'], f'{embeddingsPath}/sdxl', True)
    print('\n\n⏳ Downloading built-in upscaler...')
    builtinResources(resource['upscaler'], upscalerPath)
    print('\n\n⏳ Downloading built-in VAE...')
    builtinResources(resource['vae'], f'{vaePath}/sdxl')

# Selected resources
def selectedResources(resourceList, resourcePath):
    for resource in resourceList:
        if eval(resource['id']):
            print(f"\n {resource['name']}...")
            for url in resource['url']:
                downloader(url, resourcePath)
                print('')

def downloadSelectedResources():
    print('\n\n⏳ Downloading selected LoRA...')
    selectedResources(getResource(loraListURL)['lora'], f'{loraPath}/sdxl')
    print('\n\n⏳ Downloading selected upscaler...')
    selectedResources(getResource(upscalerListURL)['upscaler'], upscalerPath)

# Other resources
def otherResources(resourceList, resourcePath):
    for resource in resourceList:
        print('\n' + resource)
        downloader(resource, resourcePath)

def downloadOtherResources(otherLora, otherEmbeddings, otherUpscaler, otherVAE):
    if otherLora:
        print('\n\n⏳ Downloading LoRA...')
        otherResources(otherLora, f'{loraPath}/sdxl')
    if otherEmbeddings:
        print('\n\n⏳ Downloading embeddings...')
        otherResources(otherEmbeddings, f'{embeddingsPath}/sdxl')
    if otherUpscaler:
        print('\n\n⏳ Downloading upscaler...')
        otherResources(otherUpscaler, upscalerPath)
    if otherVAE:
        print('\n\n⏳ Downloading VAE...')
        otherResources(otherVAE, f'{vaePath}/sdxl')

# Remove git from directory
def removeGit(directory):
    for (folder, subfolders, files) in os.walk(directory):
        if subfolders:
            for subfolder in subfolders:
                if subfolder == '.git':
                    pathFolder = os.path.join(folder, subfolder)
                    !rm -r {pathFolder}

        for file in files:
            if file == '.gitattributes' or file == 'readme.md' or file == 'README.md':
                pathFile = os.path.join(folder, file)
                !rm {pathFile}

completedMessage()

## 2. ControlNet

### • ControlNet Models

In [ ]:
# ControlNet - https://github.com/Mikubill/sd-webui-controlnet/discussions/2039
# Select models to download. Default models are : depth, ip-adapter, openpose

controlNetXL = {  
    'blur': boolean[0],
    'blur_anime': boolean[0],
    'canny': boolean[0],
    'canny_anime': boolean[0],
    'depth': boolean[1],
    'depth_anime': boolean[0],
    'ipadapter': boolean[1],
    'lineart': boolean[0],
    'openpose': boolean[1],
    'openpose_anime': boolean[0],
    'recolor': boolean[0],
    'scribble_anime': boolean[0],
    'sketch': boolean[0],
    'softedge': boolean[0],
}


################################################################################################################

print("\n⏳ Download selected controlNet models...")
downloadControlNet(controlNetXL, controlNetURL)
os.chdir(webui)
completedMessage()

### • Install from URL

In [ ]:
# Install controlNet models from URL 
otherControlNetXL = []


########################################################################################

if otherControlNet:
    print('⏳ Downloading controlNet models...')
    otherResources(otherControlNetXL, f'{controlNetModelsPath}/sdxl')

os.chdir(webui)
completedMessage()

## 3. SDXL Models
**Choose models you want to download**  
Some models hosted by Civitai require you to login to download.  
Create an API key in [Civitai user settings](https://civitai.com/user/account) then paste it to `civiaiToken` inside Variables and Functions cell.

### • Anime / Cartoon / 3D

In [ ]:
# Animagine XL - https://civitai.com/models/260267
animagineVersions = ['Select version...', 'v3']
animagine = animagineVersions[0]

# Blue Pencil XL - https://civitai.com/models/119012
bluePencilVersions = ['Select version...', 'v5', 'Lightning v5']
bluePencil = bluePencilVersions[0]

# CounterfeitXL - https://civitai.com/models/118406
counterfeitVersions = ['Select version...', 'v2.5']
counterfeit = counterfeitVersions[0]

# DynaVision XL - https://civitai.com/models/122606
dynaVisionVersions = ['Select version...', 'v0.6.1.0_vae']
dynaVision = dynaVisionVersions[0]

# SDXL Niji - https://civitai.com/models/120765
sdxlNijiVersions = ['Select version...', 'SE']
sdxlNiji = sdxlNijiVersions[0]

# SDXL Unstable Diffusers - https://civitai.com/models/84040
unstableDiffVersions = ['Select version...', 'v11', 'v11+RunDiffusion', 'Turbo v10']
unstableDiff = unstableDiffVersions[0]


########################################################################################

downloadModels(animeModelsURL)
completedMessage()

### • General Purpose

In [ ]:
# Copax TimeLessXL - https://civitai.com/models/118111
copaxTimelessVersions = ['Select version...', 'v10', 'Turbo v1']
copaxTimeless = copaxTimelessVersions[0]

# DreamShaper XL - https://civitai.com/models/112902
dreamShaperVersions = ['Select version...', 'Lightning v1', 'Turbo DPM++ SDE v2.1']
dreamShaper = dreamShaperVersions[0]

# Juggernaut XL - https://civitai.com/models/133005
juggernautVersions = ['Select version...', 'v9', 'Lightning v9']
juggernaut = juggernautVersions[0]

# ProtoVision XL - https://civitai.com/models/125703
protoVisionVersions = ['Select version...', 'v6.6.0_vae']
protoVision = protoVisionVersions[0]

# SD XL - https://civitai.com/models/101055
sdxlVersions = ['Select version...', 'v1_vae', 'v1_vae-refiner', 'Turbo v1']
sdxl = sdxlVersions[0]


########################################################################################

downloadModels(generalModelsURL)
completedMessage()

### • Realistic

In [ ]:
# NewRealityXL - https://civitai.com/models/161068
newRealityVersions = ['Select version...', 'v2.1']
newReality = newRealityVersions[0]

# NightVision XL - https://civitai.com/models/128607
nightVisionVersions = ['Select version...', 'v0.8.1.1', 'v0.7.9.1_vae']
nightVision = nightVisionVersions[0]

# Realism Engine SDXL - https://civitai.com/models/152525
realismEngineVersion = ['Select version...', 'v3_vae']
realismEngine = realismEngineVersion[0]

# Realities Edge XL - https://civitai.com/models/129666
realEdgeVersions = ['Select version...', 'v7', 'Lightning v7', 'Turbo v7']
realEdge = realEdgeVersions[0]

# RealVisXL - https://civitai.com/models/139562
realVisVersions = ['Select version...', 'v4_vae', ' Lightning v4_vae', 'Turbo v3_vae', 'v3_vae-inpainting']
realVis = realVisVersions[0]


########################################################################################

downloadModels(realisticModelsURL)
completedMessage()

### • Woman

In [ ]:
# ArienMixXL Asian portrait - https://civitai.com/models/118913
arienmixVersions = ['Select version...', 'v4']
arienmix = arienmixVersions[0]

# LEOSAM's HelloWorld SDXL - https://civitai.com/models/43977
helloWorldVersions = ['Select version...', 'v5', 'Lightning Euler a v5', 'Turbo+LCM v2']
helloWorld = helloWorldVersions[0]

# SDVN6 RealXL - https://civitai.com/models/118114
sdvnRealVersions = ['Select version...', 'Detailface']
sdvnReal = sdvnRealVersions[0]

# XXMix 9realistic SDXL - https://civitai.com/models/124421
xxmix9realVersions = ['Select version...', 'v1']
xxmix9real = xxmix9realVersions[0]


########################################################################################

downloadModels(womanModelsURL)
completedMessage()

### • Install from URL

In [ ]:
# Install models from URL 
otherModels = []


########################################################################################

if otherModels:
    print('⏳ Downloading models...')
    otherResources(otherModels, f'{modelsPath}/sdxl')
        
os.chdir(webui)
completedMessage()

## 4. LoRA, Embedding, Upscaler and VAE

### • Built-in Resources
**This is just for information. No need to run this cell.**

**Pre-installed LoRA :**
* [Detail Tweaker XL](https://civitai.com/models/122359)

**Pre-installed Embeddings :**
* [Negative Prompts Pack XL](https://huggingface.co/ffxvs/negative-prompts-pack-xl)
* [Embeddings Collection XL](https://huggingface.co/ffxvs/embeddings-collection-xl)

**Pre-installed Upscaler :**  
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**
* [sdxl-vae-fp16-fix](https://huggingface.co/madebyollin/sdxl-vae-fp16-fix)

### • Resource Lists

In [ ]:
####################################### LORA ##########################################

# Blur SDXL - https://civitai.com/models/124855
blurXL = boolean[0]

# DetailedEyes XL - https://civitai.com/models/120723
detailEyes = boolean[0]

# LCM Lora SDXL - https://huggingface.co/latent-consistency/lcm-lora-sdxl
lcmXL = boolean[0]

# LCM&TurboMix - https://civitai.com/models/216190
lcmTurbo = boolean[0]

# Hands XL - https://civitai.com/models/200255
handsXL = boolean[0]

# XL More Art Full - https://civitai.com/models/124347
moreArt = boolean[0]


###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[0]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]


###################################################################################################

os.chdir(webui)
downloadBuiltinResources()
downloadSelectedResources()
os.chdir(webui)
completedMessage()

### • Install from URL

In [ ]:
# LoRA
otherLora = []

# Embeddings
otherEmbeddings = []

# Upscaler
otherUpscaler = []

# VAE
otherVAE = []


###################################################################################################

downloadOtherResources(otherLora, otherEmbeddings, otherUpscaler, otherVAE)
os.chdir(webui)
completedMessage()